In [1]:
import pandas as pd
import numpy as np

# Import all files

In [2]:
invoice = pd.read_excel("Courier Company - Invoice.xlsx")
order = pd.read_excel("Company X - Order Report.xlsx")
rates = pd.read_excel("Courier Company - Rates.xlsx")
pincode = pd.read_excel("Company X - Pincode Zones.xlsx")
sku = pd.read_excel("Company X - SKU Master.xlsx")

# Merging files to calculate total weight as per company X

In [3]:
order_sku_merge = pd.merge(order,sku, on='SKU')
order_sku_merge['Total weight as per X (KG)'] = (order_sku_merge['Order Qty'] * order_sku_merge['Weight (g)'])/1000
order_sku_merge= order_sku_merge.rename(columns={'ExternOrderNo': 'Order ID'})
total_weight = order_sku_merge.groupby('Order ID').agg({
    'Total weight as per X (KG)': 'sum',
    'Order Qty': 'sum',
    'Weight (g)': 'sum'
}).reset_index()
total_weight

,Order ID,Total weight as per X (KG),Order Qty,Weight (g)
0,2001806210,0.220,2.0,220
1,2001806226,0.480,2.0,240
2,2001806229,0.500,3.0,500
3,2001806232,1.302,8.0,377
4,2001806233,0.245,2.0,245
...,...,...,...,...
119,2001821995,0.477,2.0,477
120,2001822466,1.376,10.0,352
121,2001823564,0.672,6.0,336
122,2001825261,1.557,13.0,611


In [4]:
invoice = invoice.rename(columns={'Zone': 'Delivery Zone charged by Courier Company'})
temp_pin = pincode.drop_duplicates(subset=['Customer Pincode'])
pin_inv_merge = invoice.merge(temp_pin, on= ['Customer Pincode', 'Warehouse Pincode'])
pin_inv_merge

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Billing Amount (Rs.),Zone
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,d
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,d
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,b
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,d
...,...,...,...,...,...,...,...,...,...
119,1091117904860,2001811039,0.68,121003,302020,d,Forward charges,90.2,b
120,1091121846136,2001811305,0.50,121003,302020,d,Forward charges,45.4,b
121,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,b
122,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,b


# Assigning rates to company x data as per courier company

In [5]:
rates['Zone']= rates['Zone'].str.lower()
company_rates = pd.merge(pin_inv_merge, rates, on= 'Zone', how= 'inner')
company_rates

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Billing Amount (Rs.),Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d,1.5,45.4,44.8,41.3,44.8
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,d,1.5,45.4,44.8,41.3,44.8
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,d,1.5,45.4,44.8,41.3,44.8
3,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8
4,1091117229555,2001806686,0.15,121003,326502,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118004245,2001811604,0.80,121003,173212,b,Forward charges,61.3,e,2.0,56.6,55.5,50.7,55.5
120,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5
121,1091120352712,2001819252,0.30,121003,174101,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5
122,1091122418320,2001827036,1.60,121003,173213,b,Forward charges,117.9,e,2.0,56.6,55.5,50.7,55.5


In [6]:
company_data = pd.merge(company_rates, total_weight, on='Order ID')
company_data= company_data.rename(columns={'Charged Weight': 'Total weight as per Courier Company (KG)', 'Billing Amount (Rs.)': 'Charges Billed by Courier Company (Rs.)'})
company_data

,AWB Code,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Total weight as per X (KG),Order Qty,Weight (g)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d,1.5,45.4,44.8,41.3,44.8,1.302,8.0,377
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,d,1.5,45.4,44.8,41.3,44.8,0.615,4.0,615
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,d,1.5,45.4,44.8,41.3,44.8,2.265,12.0,929
3,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8,0.240,1.0,240
4,1091117229555,2001806686,0.15,121003,326502,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8,0.240,1.0,240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118004245,2001811604,0.80,121003,173212,b,Forward charges,61.3,e,2.0,56.6,55.5,50.7,55.5,0.721,4.0,481
120,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,0.689,4.0,689
121,1091120352712,2001819252,0.30,121003,174101,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,0.270,3.0,270
122,1091122418320,2001827036,1.60,121003,173213,b,Forward charges,117.9,e,2.0,56.6,55.5,50.7,55.5,2.176,10.0,2039


# Assigning weight slabs

In [7]:
def slab(weight):
    i = round(weight%1,1)
    if i ==0.0:
        return weight
    elif i >0.5:
        return int(weight)+1.0
    else:
        return int(weight) + 0.5
    

In [8]:
company_data['Weight slab as per X (KG)'] = company_data['Total weight as per X (KG)'].apply(slab)
company_data['Weight slab charged by Courier Company (KG)'] = company_data['Total weight as per Courier Company (KG)'].apply(slab)
company_data= company_data.rename(columns={'Zone':'Delivery Zone as per X', 'AWB Code': 'AWB Number'})
company_data

,AWB Number,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Delivery Zone as per X,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Total weight as per X (KG),Order Qty,Weight (g),Weight slab as per X (KG),Weight slab charged by Courier Company (KG)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d,1.5,45.4,44.8,41.3,44.8,1.302,8.0,377,1.5,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,d,1.5,45.4,44.8,41.3,44.8,0.615,4.0,615,1.0,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,d,1.5,45.4,44.8,41.3,44.8,2.265,12.0,929,2.5,2.5
3,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8,0.240,1.0,240,0.5,0.5
4,1091117229555,2001806686,0.15,121003,326502,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8,0.240,1.0,240,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118004245,2001811604,0.80,121003,173212,b,Forward charges,61.3,e,2.0,56.6,55.5,50.7,55.5,0.721,4.0,481,1.0,1.0
120,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,0.689,4.0,689,1.0,0.5
121,1091120352712,2001819252,0.30,121003,174101,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,0.270,3.0,270,0.5,0.5
122,1091122418320,2001827036,1.60,121003,173213,b,Forward charges,117.9,e,2.0,56.6,55.5,50.7,55.5,2.176,10.0,2039,2.5,2.0


# Calculating Expected charge as per company x

In [9]:
expected_charge = []
for index, row in company_data.iterrows():
    weight = row['Weight slab as per X (KG)']
    shipment = row['Type of Shipment']
    forward_charge = row['Forward Fixed Charge']
    additional_charge = row['Forward Additional Weight Slab Charge']
    rto_charge = row['RTO Fixed Charge']
    rto_additional = row['RTO Additional Weight Slab Charge']
    
    if shipment == 'Forward charges':
        if weight <= 0.5:
            expected_charge.append(forward_charge)
        else:
            additional_weight = (weight -0.5)/0.5
            expected_charge.append(forward_charge +(additional_charge * additional_weight))
    elif shipment == 'Forward and RTO charges':
        if weight <= 0.5:
            expected_charge.append(forward_charge + rto_charge)
        else:
            additional_weight = (weight - 0.5)/0.5
            expected_charge.append(forward_charge +(additional_charge * additional_weight)+ rto_charge +(rto_additional * additional_weight))
    else:
        None

company_data['Expected Charge as per X (Rs.)'] = expected_charge

In [10]:
company_data

,AWB Number,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Delivery Zone as per X,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Total weight as per X (KG),Order Qty,Weight (g),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Expected Charge as per X (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d,1.5,45.4,44.8,41.3,44.8,1.302,8.0,377,1.5,1.5,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,d,1.5,45.4,44.8,41.3,44.8,0.615,4.0,615,1.0,1.0,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,d,1.5,45.4,44.8,41.3,44.8,2.265,12.0,929,2.5,2.5,224.6
3,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8,0.240,1.0,240,0.5,0.5,45.4
4,1091117229555,2001806686,0.15,121003,326502,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8,0.240,1.0,240,0.5,0.5,45.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118004245,2001811604,0.80,121003,173212,b,Forward charges,61.3,e,2.0,56.6,55.5,50.7,55.5,0.721,4.0,481,1.0,1.0,112.1
120,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,0.689,4.0,689,1.0,0.5,112.1
121,1091120352712,2001819252,0.30,121003,174101,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,0.270,3.0,270,0.5,0.5,56.6
122,1091122418320,2001827036,1.60,121003,173213,b,Forward charges,117.9,e,2.0,56.6,55.5,50.7,55.5,2.176,10.0,2039,2.5,2.0,278.6


# Calculating fair Charge status

In [11]:
fair_charge = []
for index, row in company_data.iterrows():
    
    if row['Charges Billed by Courier Company (Rs.)'] > row['Expected Charge as per X (Rs.)']:
        fair_charge.append('overcharged')
    elif row['Charges Billed by Courier Company (Rs.)'] == row['Expected Charge as per X (Rs.)']:
        fair_charge.append('correctly charged')
    else:
        fair_charge.append('undercharged')
        
        
company_data['charged status'] = fair_charge

In [12]:
company_data = company_data.drop(['Warehouse Pincode','Customer Pincode', 'Weight (g)'],axis=1)
company_data['charged status'].value_counts()

charged status
overcharged          86
undercharged         20
correctly charged    18
Name: count, dtype: int64

In [13]:
company_data

,AWB Number,Order ID,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Delivery Zone as per X,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Total weight as per X (KG),Order Qty,Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Expected Charge as per X (Rs.),charged status
0,1091117222124,2001806232,1.30,d,Forward charges,135.0,d,1.5,45.4,44.8,41.3,44.8,1.302,8.0,1.5,1.5,135.0,correctly charged
1,1091117222194,2001806273,1.00,d,Forward charges,90.2,d,1.5,45.4,44.8,41.3,44.8,0.615,4.0,1.0,1.0,90.2,overcharged
2,1091117222931,2001806408,2.50,d,Forward charges,224.6,d,1.5,45.4,44.8,41.3,44.8,2.265,12.0,2.5,2.5,224.6,correctly charged
3,1091117229345,2001807012,0.15,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8,0.240,1.0,0.5,0.5,45.4,correctly charged
4,1091117229555,2001806686,0.15,d,Forward charges,45.4,d,1.5,45.4,44.8,41.3,44.8,0.240,1.0,0.5,0.5,45.4,correctly charged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118004245,2001811604,0.80,b,Forward charges,61.3,e,2.0,56.6,55.5,50.7,55.5,0.721,4.0,1.0,1.0,112.1,undercharged
120,1091121844806,2001811475,0.50,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,0.689,4.0,1.0,0.5,112.1,undercharged
121,1091120352712,2001819252,0.30,b,Forward charges,33.0,e,2.0,56.6,55.5,50.7,55.5,0.270,3.0,0.5,0.5,56.6,undercharged
122,1091122418320,2001827036,1.60,b,Forward charges,117.9,e,2.0,56.6,55.5,50.7,55.5,2.176,10.0,2.5,2.0,278.6,undercharged


# Sum of overcharged, undercharges & correctly charge

In [14]:
company_data['Difference Between Expected Charges and Billed Charges (Rs.)'] = (company_data['Expected Charge as per X (Rs.)'] - company_data['Charges Billed by Courier Company (Rs.)']).apply(lambda x: '{:.2f}'.format(x))
company_data = company_data.drop(['Type of Shipment','Weight Slabs','Forward Fixed Charge','RTO Fixed Charge','RTO Additional Weight Slab Charge','Forward Additional Weight Slab Charge','Order Qty'],axis=1)
company_data

,AWB Number,Order ID,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Delivery Zone as per X,Total weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Expected Charge as per X (Rs.),charged status,Difference Between Expected Charges and Billed Charges (Rs.)
0,1091117222124,2001806232,1.30,d,135.0,d,1.302,1.5,1.5,135.0,correctly charged,0.00
1,1091117222194,2001806273,1.00,d,90.2,d,0.615,1.0,1.0,90.2,overcharged,-0.00
2,1091117222931,2001806408,2.50,d,224.6,d,2.265,2.5,2.5,224.6,correctly charged,0.00
3,1091117229345,2001807012,0.15,d,45.4,d,0.240,0.5,0.5,45.4,correctly charged,0.00
4,1091117229555,2001806686,0.15,d,45.4,d,0.240,0.5,0.5,45.4,correctly charged,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118004245,2001811604,0.80,b,61.3,e,0.721,1.0,1.0,112.1,undercharged,50.80
120,1091121844806,2001811475,0.50,b,33.0,e,0.689,1.0,0.5,112.1,undercharged,79.10
121,1091120352712,2001819252,0.30,b,33.0,e,0.270,0.5,0.5,56.6,undercharged,23.60
122,1091122418320,2001827036,1.60,b,117.9,e,2.176,2.5,2.0,278.6,undercharged,160.70


In [16]:
company_data['Difference Between Expected Charges and Billed Charges (Rs.)'] = pd.to_numeric(company_data['Difference Between Expected Charges and Billed Charges (Rs.)'], errors='coerce')


In [17]:
company_data.loc[company_data['charged status'] == 'overcharged', 'Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

-4572.469999999999

In [18]:
company_data.loc[company_data['charged status'] == 'correctly charged', 'Charges Billed by Courier Company (Rs.)'].sum()

1493.1999999999998

In [19]:
company_data.loc[company_data['charged status']=='undercharged', 'Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

465.65000000000003

In [20]:
company_data= company_data.drop(['charged status'],axis=1)

# Reindexing Columns

In [22]:
new_column_order = ['Order ID', 'AWB Number', 'Total weight as per X (KG)', 'Weight slab as per X (KG)', 
                    'Total weight as per Courier Company (KG)', 'Weight slab charged by Courier Company (KG)', 
                    'Delivery Zone as per X', 'Delivery Zone charged by Courier Company', 
                    'Expected Charge as per X (Rs.)', 'Charges Billed by Courier Company (Rs.)', 
                    'Difference Between Expected Charges and Billed Charges (Rs.)']

In [23]:
company_data = company_data.reindex(columns=new_column_order)

In [24]:
company_data

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,-0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
4,2001806686,1091117229555,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,0.721,1.0,0.80,1.0,e,b,112.1,61.3,50.8
120,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0,79.1
121,2001819252,1091120352712,0.270,0.5,0.30,0.5,e,b,56.6,33.0,23.6
122,2001827036,1091122418320,2.176,2.5,1.60,2.0,e,b,278.6,117.9,160.7


# Save the file in new excel sheet

In [31]:
company_data.to_excel('Cointab_Assignment.xlsx', index=False)

In [32]:
pd.read_excel('Cointab_Assignment.xlsx')

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
4,2001806686,1091117229555,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,0.721,1.0,0.80,1.0,e,b,112.1,61.3,50.8
120,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0,79.1
121,2001819252,1091120352712,0.270,0.5,0.30,0.5,e,b,56.6,33.0,23.6
122,2001827036,1091122418320,2.176,2.5,1.60,2.0,e,b,278.6,117.9,160.7
